In [1]:
import os
import sys
from zipfile import ZipFile
from dotenv import load_dotenv
from langchain_chroma import Chroma

In [3]:
load_dotenv('ya.env')

True

In [4]:
arch_p = os.environ['archive_path']
proj_p = os.environ['proj_path']
module_p = os.environ['module_p']
source_p = os.environ['source_p']

In [5]:
os.listdir(proj_p)

['RAG.ipynb', 'ya.env', 'little_halper.ipynb']

In [6]:

# zf = ZipFile(os.path.join(arch_p,'chroma_db.zip'), 'r')
# zf.extractall(proj_p)
# zf.close()

In [7]:
# zf = ZipFile(os.path.join(arch_p,'chroma_hypoq_db.zip'), 'r')
# zf.extractall(proj_p)
# zf.close()

In [8]:
sys.path.append(module_p)

In [9]:
from bot.preproc_txt import normalize

In [10]:
os.listdir(source_p)

['wine.owl', 'wine.rdf', 'regions.txt', 'wines.txt', 'food.owl', 'food.rdf']

In [11]:
with open(os.path.join(source_p,'wines.txt'),encoding='utf-8') as f:
    wines = ''.join(f.readlines())
with open(os.path.join(source_p,'regions.txt'),encoding='utf-8') as f:
    regions = ''.join(f.readlines())

wines = [normalize(x) for x in wines.split('-----')]
regions = [normalize(x) for x in regions.split('-----')]

In [12]:
wines[0]

'Каберне фран — темный сорт винограда, один из древнейших мировых сортов, от которого произошли такие сорта как: каберне совиньон, карменер, мерло. Фран в названии обозначает французский. Происхождение слова каберне имеет разные версии, однако подлинность их не установлена. Используется для производства красных и розовых вин, как моносортовых, так и в составе бленда из нескольких сортов.\nНекоторые исследования указывают на то, что предок сорта каберне фран завезен во Францию из Испании. Существуют отдельные указания на то, что каберне фран выращивали на виноградниках долины Луары еще в XIV веке. Однако чаще можно встретить версию, что этот сорт начали активно и повсеместно высаживать во Франции в XVII веке.\nКаберне фран часто используют для создания ассамбляжа красных вин. Процент этого сорта в составе бленда с мерло и каберне совиньон может быть как минимальным, так и достигать 50% от общего объема.\nКрасные вина каберне фран отличаются характерным ароматом, в котором присутствуют т

In [13]:
max(len(x) for x in wines+regions)

2852

In [14]:
import os
from yandex_chain import YandexEmbeddings

embeddings = YandexEmbeddings(
    folder_id=os.environ['folder_id'], 
    api_key=os.environ['api_key'])
 

# vec = embeddings.embed_query("Hello, world!")
# len(vec)

In [16]:
db = Chroma(embedding_function=embeddings, persist_directory='./chroma_db')

In [17]:
q = "Что едят с мерло?"
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 5})
res = retriever.invoke(q)
res

[Document(page_content='\nМерло\nМерло (фр. Merlot) — сорт винного красного винограда из Бордо, винодельческого региона Франции. Название сорта происходит от merle (фр.) — черный дрозд, обозначая сходство темного оттенка ягод с оперением птиц. Мерло растет в умеренном и теплом климате и устойчив к холодам. Этот сорт прост в уходе, обладает высокой урожайностью, что делает его вторым по распространенности красным виноградом в мире — после каберне совиньон. Мерло выращивают во Франции, США, Италии и других странах.\nСамое раннее упоминание мерло относится к 1784 году: чиновник из Бордо назвал вино из этого сорта лучшим из представленных правобережными виноградниками Либурне. К XIX веку виноград регулярно сажали в соседнем северном регионе Медок на левом берегу Жиронды.\nИз мерло получаются мягкие вина с низкой кислотностью, минимальным содержанием танинов, бархатистым вкусом. Мерло используют в красных и розовых блендах вин, чтобы сбалансировать сухость, крепость и терпкие танинные свойс

In [24]:
from bot.llm_chain import chain

In [25]:
chain.invoke('Какие существуют итальянские вина?')

'Существует множество итальянских вин, среди которых — амароне, соаве, просекко, речото и вина из местности Вальполичелла (регион Венето), бароло и барбареско (Пьемонт), марсала (Сицилия), биферно (Молизе) и альянико дель табурно, греко ди туфо и фьяно ди авеллино (Кампания). Это лишь некоторые из множества производимых в Италии вин.'